In [88]:
#Python script that collects top 5 popular videos of every user that ranked in top 100 and post data of his timeline
import re
import json
import sys
import os
import pickle
import shutil


posts = "../vinedata/posts.txt"
videos ="../vinedata/videos.txt"
users = "../vinedata/users.txt"

copyRoot = "/datasets/sagarj/vineData/Dataset/"
srcRoot = "/datasets/sagarj/vineData/"

videoDir = copyRoot+"Videos/"
postDir = copyRoot+"Posts/"
savedPostsDir = copyRoot + "savedPosts/"
userDir = copyRoot+"Users/"

visitedList = "../Logs/crawledUsers.pk"
postVisited = "../Logs/crawledPosts.pk"

userList = copyRoot + "users.pk"
post_video_list = copyRoot + "dictionary.pk"



In [89]:
def getVisited(visitedList):
    visited = []
    try:
        f = open(visitedList, 'rb')
        visited = pickle.load(f)
    except IOError:
        f = open(visitedList,"a+")
        pickle.dump([],f)
    return visited


def updateVisited(visited,visitedList):
    with open(visitedList, 'wb') as f:
        pickle.dump(visited, f)


def walkLevel1Dir(root):
    count = 0
    dirList = []
    filesList = []
    for path, dirs, files in os.walk(root):
        if count > 0:
            return dirList , fileList
        dirList = dirs
        fileList = files
        count = count + 1


def getPopularFile(rootDir):
    f = open(rootDir + '/popular.json' ,'r')
    data = json.load(f)
    return data

def getJson(jsonFile):
    f = open(jsonFile ,'r')
    data = json.load(f)
    return data

# Get Top 3 liked videos
def getVideos(usrProf, userId, vid ):
    videoDir = vid + "/" + str(userId)
    print "Working on Vids:  "  + videoDir
    if not os.path.exists(videoDir):
        os.makedirs(videoDir)
    if len(usrProf) < 2:
        return
    if not isinstance(usrProf['data'],dict):
        print "JSON Data not a dict"
        return
    records = usrProf['data']['records']
    likes = []
    for record in records:
        likes.append(record['likes']['count'])
    likes.sort(reverse=True)
    
    for record in records:
        if record['likes']['count'] in likes[:3]:
            videoString = record['videoDashUrl']
            if videoString:
                videoUrl = record['videoDashUrl'].split('?');
                argsVideo = ['wget', '-r', '-l', '1', '-p', '-P' ,'-nd', videoDir, videoUrl[0]]
                call(argsVideo);

def getPostMeta(usrProf, userId , post ):
    postDir = post + "/" + str(userId)
    print "Working on Post:  "  + postDir
    if not os.path.exists(postDir):
        os.makedirs(postDir)
    if not isinstance(usrProf['data'],dict):
        print "JSON Data not a dict"
        return
    records = usrProf['data']['records']
 
    for subRecord in records:
        postID = subRecord['postId']
        postData = requests.get("https://api.vineapp.com/timelines/posts/"+str(postID))
        post = open( postDir + "/" + str(postID) + '.json', 'w')
        json.dump(postData.json(), post)

def getHistory(popular , dataRoot):
    records = popular['data']['records']
    timeline = dataRoot + "/" + userPastVideos
    posts = dataRoot + "/" + userPastVideoMeta
    if not os.path.exists(timeline):
        os.makedirs(timeline)
    if not os.path.exists(posts):
        os.makedirs(posts)
    print "now in : " + dataRoot
    for record in records:
        userId = record['userId']
        profile_file = dataRoot + "/userTimeline/" + str(userId) + ".json"
        print "Working on file:  "  + profile_file
        if os.path.exists(profile_file):
            usrProf = getJson(profile_file)
            getVideos(usrProf,userId, timeline)
            getPostMeta(usrProf,userId,posts)

def getList(textFile):
    lines = []
    ids = []
    with open(textFile) as f:
        lines = f.readlines()
    for line in lines:
        parts = line.split('/')
        ids.append(parts[-1])
    unique = list(set(ids))
    print len(unique)
    ulines = []
    for i in unique:
        ulines.append(lines[ids.index(i)].strip())
    print "%d reduced to %d"%(len(lines), len(ulines)) 
    return ulines

def copyFile(src, dest):
    if not os.path.exists(src): 
        return
    try:
        shutil.copy(src, dest)
    # eg. src and dest are the same file
    except shutil.Error as e:
        print('Error: %s' % e)
    # eg. source or destination doesn't exist
    except IOError as e:
        print('Error: %s' % e.strerror)

In [5]:
postList = getList(posts)

89150
974746 reduced to 89150


In [13]:

allPosts = []
allPostNames = []
for p in postList:
    name = p.split('/')[-1]
    allPostNames.append(name)
    postJson = postDir + name
    f = open(postJson,'r')
    data = json.load(f)
    allPosts.append(data)

In [1]:
print allPosts[10000]['data']['records'][0]['postId']

NameError: name 'allPosts' is not defined

path =  srcRoot + postList[1]
print path
os.path.lexists(path.strip())

In [44]:
allVideos = ["" for x in range(len(allPosts))]
for i in range(len(allPosts)):
    allVideos[i] = ""
    if isinstance(allPosts[i]['data'],dict) and isinstance(allPosts[i]['data']['records'][0],dict):
        videoUrl = allPosts[i]['data']['records'][0]['videoDashUrl']
        if isinstance(videoUrl, basestring):
            allVideos[i] = videoUrl.split('?')[0].split('/')[-1]
    
print len(allVideos)

89150


print len(postList)
for src in postList:
    path =  srcRoot + src
    copyFile(path.strip(),postDir)

In [72]:
userList = getList(users)

1909
24234 reduced to 1909


In [78]:
users = []
for u in userList:
    fileName = u.split('/')[-1]
    users.append(fileName.split('.')[0])

In [79]:
print users

['1251414986228908032', '1114824010501586944', '933124602639552512', '1273461841284476928', '908324861313683456', '1249169371285389312', '934268079968509952', '906798754340937728', '1135668670555709440', '915433899604197376', '906726600555511808', '952325950346018816', '1009540564544528384', '991200655933128704', '931277886948782080', '1174092642494402560', '984750104915632128', '1230358474174164992', '940134498446770176', '1086709285821575168', '908404092668350464', '1075621962124062720', '1107134867197841408', '933919962593124352', '954584885736128512', '1224962973584535552', '1166300183546503168', '1120872942210056192', '906282604641267712', '989785170108694528', '1241271220939972608', '912986914137706496', '906816615973261312', '913899469181956096', '1197012837638217728', '984374458486022144', '940045338654425088', '1122929066777997312', '1157114243942932480', '1248880963354767360', '1167992299910004736', '921487719098626048', '932909857579872256', '957122715427454976', '9390260964

In [80]:
#updateVisited(users, visitedList)


print userList[10]
path =  srcRoot + userList[10]
print path
os.path.lexists(path.strip())

print len(userList)
for src in userList:
    path =  srcRoot + src
    copyFile(path.strip(),userDir)

In [6]:
videoList = getList(videos)

12369
79422 reduced to 12369


print videoList[1]

path =  srcRoot + videoList[1]
print path
os.path.exists(path.strip())

print len(videoList)
for src in videoList:
    path =  srcRoot + src
    copyFile(path.strip(),videoDir)

In [47]:
print userList[1]

Data/1451152801/userTimeline/1114824010501586944.json


In [48]:
savedVids = []
for v in videoList:
    savedVids.append(v.split('/')[-1])

In [60]:
savedPosts = []
failedPosts = []
for vid in savedVids:
    try:
        index = allVideos.index(vid)
        savedPosts.append(allPostNames[index])
    except ValueError:
        failedPosts.append(vid)
        #print "Threw value error at %s" %vid
        
    

In [82]:
savedPostIds = []
for p in savedPosts:
    savedPostIds.append(p.split('.')[0])
print savedPostIds[0]

'1271528897837694976.json'

In [62]:
print len(videoList)

12369


In [63]:
# for p in savedPosts: 
#     srcFile = (postDir + p).strip()
#     copyFile(srcFile, savedPostsDir)

In [64]:
allPosts = []
for p in savedPosts:
    srcFile = (savedPostsDir + p).strip()
    f = open(srcFile,'r')
    data = json.load(f)
    allPosts.append(data)

In [66]:
print allPosts[1]

{u'code': u'', u'data': {u'count': 1, u'anchorStr': u'', u'records': [{u'promoted': 0, u'liked': 0, u'videoDashUrl': u'http://mtc.cdn.vine.co/r/videos_dashhd/A55FD9E26D1282171909374984192_10df7abc2d3.3.2.mp4?versionId=LH3uNzFev2tnMO2OXzCFziknEF3VQjZe', u'foursquareVenueId': u'', u'userId': 1167288945915121664, u'private': 0, u'videoWebmUrl': None, u'loops': {u'count': 99184.0, u'velocity': 0.0, u'onFire': 0}, u'thumbnailUrl': u'http://v.cdn.vine.co/r/thumbs/65B465B0C41282171910347751424_1.3.2.mp4.jpg?versionId=PXOBpNaCvAALUuhjEMupzP.W0Q.KT4_f', u'explicitContent': 0, u'description': u'I will never understand his goofiness\U0001f602\n\nac: crazed audios', u'verified': 0, u'avatarUrl': u'http://v.cdn.vine.co/r/avatars/662C7B115C1338660082258681856_51936b4d25c.13.0.jpg?versionId=rAyWQJb5maSngVuZEYCXSo7_VzsjvBPT', u'comments': {u'count': 84, u'anchorStr': u'1316993391418073088', u'records': [{u'username': u'Tracy', u'comment': u'MMMMMHHHHMMMMMM \U0001f60f\U0001f60f\U0001f60f\U0001f60f Magc